In [ ]:
# Code to download observations of lichens from iNaturalist
# https://github.com/pyinat/pyinaturalist

In [ ]:
# Install pyinaturalist
%pip install pyinaturalist

In [ ]:
# Import the main API functions
from pyinaturalist import *

In [ ]:
# install pandas
%pip install pandas

In [ ]:
import pandas as pd
print(pd.__version__)

In [ ]:
# The fields we are interested in searching are Taxon and 
# List of other fields that can be used https://pyinaturalist.readthedocs.io/en/stable/modules/pyinaturalist.v1.observations.html#pyinaturalist.v1.observations.create_observation

In [11]:
from pyinaturalist import get_observations, pprint
observations = get_observations(taxon_id = 54743,
                                place_guess = 'Texas, USA',
                                quality_grade = 'research',
                                page = 'all')
pprint(observations)

This request is larger than recommended for API usage. For bulk requests, consider using the iNat export tool instead: https://www.inaturalist.org/observations/export


ConnectionError: HTTPSConnectionPool(host='api.inaturalist.org', port=443): Max retries exceeded with url: /v1/observations?quality_grade=research&taxon_id=54743&place_guess=Texas%2C+USA&id_above=5156721&per_page=200&order_by=id&order=asc (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x3613556d0>: Failed to resolve 'api.inaturalist.org' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
import os
import time
import requests
from pyinaturalist import get_observations

# Specify the taxon ID for lichens (54743)
taxon_id = 54743

# Specify the place (Texas, USA)
place_guess = "Texas, USA"

# Directory to save images
save_dir = '/Users/eabowman/Dropbox/LichenProject/iNaturalist'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Function to download image with retry
def download_image(image_url, save_path, retries=3, delay=60):
    for attempt in range(retries):
        try:
            # Request the image
            response = requests.get(image_url)
            response.raise_for_status()  # Raise an error if the request fails

            # Write the image to file
            with open(save_path, 'wb') as file:
                file.write(response.content)
            print(f"Downloaded {save_path}")
            return  # Successfully downloaded, exit function
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed for {image_url}: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)  # Wait before retrying
            else:
                print(f"Failed to download image after {retries} attempts.")

# Function to get observations with retry
def get_observations_with_retry(taxon_id, place_guess, retries=3, delay=60):
    for attempt in range(retries):
        try:
            response = get_observations(
                taxon_id=taxon_id,
                place_guess=place_guess,
                photos=True,  # Fetch observations with photos
                per_page=50,  # Adjust per_page as needed (max 200)
                page=1  # Page number, iterate if needed
            )
            return response
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for fetching observations: {e}")
            if attempt < retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)  # Wait before retrying
            else:
                raise e

# Fetch observations with retry
response = get_observations_with_retry(taxon_id, place_guess)


# Debugging: print the first few observations to check their data
print("First few observations:", response['results'][:5])

# Loop through the results and download the images
for observation in response['results']:
    # Debugging: check if the observation is tagged as lichen
    print(f"Observation ID: {observation['id']}, Species: {observation.get('species_guess', 'Unknown')}")
    
    # Check if the observation has photos
    if 'photos' in observation:
        for photo in observation['photos']:
            # Get the full-resolution image URL (original image, not a thumbnail)
            image_url = photo['url']
            if 'original_dimensions' in photo:  # If larger image exists
                image_url = photo['original_dimensions']['url']  # Full-resolution URL
            
            # Create a filename for the image
            image_filename = os.path.join(save_dir, f"obs_{observation['id']}_photo_{photo['id']}.jpg")
            
            # Download the image with retry
            download_image(image_url, image_filename)

In [ ]:
# Get an access token
from pyinaturalist.auth import get_access_token

# Fill in the values from your iNaturalist app
access_token = get_access_token(
    username='lizbowman',
    password='BowMan2#',
    app_id='YOUR_CLIENT_ID',
    app_secret='YOUR_CLIENT_SECRET'
)

print(access_token)

In [ ]:
# Set up access token
from pyinaturalist import set_access_token, get_observations

# Set your access token
set_access_token('YOUR_ACCESS_TOKEN')

In [ ]:
import json
import os
import requests
from pyinaturalist import get_observations

# Set the directory where you'd like to save the data
output_dir = '/Users/eabowman/Dropbox/LichenProject/iNaturalist'
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# File paths for saving observations and images
observations_file = os.path.join(output_dir, 'lichens_observations.json')
images_dir = os.path.join(output_dir, 'images')
os.makedirs(images_dir, exist_ok=True)  # Create an 'images' subdirectory for images

taxon_id = 48460  # Lichens
all_results = []

place_guess = 'Texas, USA'  # Use Texas as the place

page = 1
while True:
    response = get_observations(
        taxon_id=taxon_id,
        place_guess=place_guess,  # Specify Texas as the place
        photos=True,
        per_page=200,
        page=page
    )
    results = response['results']
    if not results:
        break
    all_results.extend(results)
    print(f'Fetched page {page} with {len(results)} records')
    page += 1

# Save observations to a JSON file
with open(observations_file, 'w') as f:
    json.dump(all_results, f, indent=4)

print(f"Saved {len(all_results)} observations to {observations_file}")

# Function to download images
def download_images(observations, images_dir):
    for observation in observations:
        photos = observation.get('photos', [])
        for i, photo in enumerate(photos):
            image_url = photo.get('url')
            if image_url:
                try:
                    image_name = f"obs_{observation['id']}_photo_{i + 1}.jpg"
                    image_path = os.path.join(images_dir, image_name)
                    
                    # Download the image
                    image_data = requests.get(image_url).content
                    
                    # Save the image to the directory
                    with open(image_path, 'wb') as img_file:
                        img_file.write(image_data)
                    
                    print(f"Downloaded image {image_name}")
                except Exception as e:
                    print(f"Error downloading image for observation {observation['id']}: {e}")

# Download images for all observations
download_images(all_results, images_dir)
